<u>**Part B. Normalize the data (5 marks)**</u>

Repeat Part A but use a normalized version of the data:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data [concrete_data_columns [concrete_data_columns  != 'Strength']] # all columns except Strength
target = concrete_data ['Strength'] # Strength column

<a id="item2"></a>


Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [8]:
predictors_norm.shape

(1030, 8)

Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [9]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


#### Let's go ahead and import the Keras library


In [10]:
import keras
import tensorflow as tf

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3507)])
    except RuntimeError as e:
        print(e)

2022-07-20 23:55:15.010130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 23:55:15.050017: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 23:55:15.050240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [11]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    # one hidden layer of 10 nodes, and a ReLU activation function
    # use the adam optimizer and the mean squared error as the loss function
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [13]:
from sklearn.model_selection import train_test_split

# step 1
# randomly split the data into a training and test sets by holding 30% of the data for testing
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target.to_numpy(), test_size=0.30)

Let's call the function now to create our model.

In [14]:
# build the model
model = regression_model()

2022-07-20 23:55:15.482720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 23:55:15.482962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 23:55:15.483130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 23:55:16.012747: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 23:55:16.012964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [15]:
from Sequences import DatasetGenerator

# data generator with tensorflow on the GPU
train_set = DatasetGenerator(X_train, y_train, 3507)
test_set = DatasetGenerator(X_test, y_test, 3507)

# step 2
# train the model on the training data using 50 epochs
model.fit(train_set, validation_data=test_set, epochs=50, verbose=2)

Epoch 1/50
1/1 - 1s - loss: 1619.9996 - val_loss: 1534.5955 - 912ms/epoch - 912ms/step
Epoch 2/50
1/1 - 0s - loss: 1619.3536 - val_loss: 1533.9578 - 56ms/epoch - 56ms/step
Epoch 3/50
1/1 - 0s - loss: 1618.7086 - val_loss: 1533.3215 - 61ms/epoch - 61ms/step
Epoch 4/50
1/1 - 0s - loss: 1618.0647 - val_loss: 1532.6858 - 58ms/epoch - 58ms/step
Epoch 5/50
1/1 - 0s - loss: 1617.4215 - val_loss: 1532.0509 - 51ms/epoch - 51ms/step
Epoch 6/50
1/1 - 0s - loss: 1616.7792 - val_loss: 1531.4172 - 60ms/epoch - 60ms/step
Epoch 7/50
1/1 - 0s - loss: 1616.1373 - val_loss: 1530.7845 - 62ms/epoch - 62ms/step
Epoch 8/50
1/1 - 0s - loss: 1615.4963 - val_loss: 1530.1530 - 54ms/epoch - 54ms/step
Epoch 9/50
1/1 - 0s - loss: 1614.8569 - val_loss: 1529.5221 - 53ms/epoch - 53ms/step
Epoch 10/50
1/1 - 0s - loss: 1614.2183 - val_loss: 1528.8916 - 59ms/epoch - 59ms/step
Epoch 11/50
1/1 - 0s - loss: 1613.5804 - val_loss: 1528.2627 - 58ms/epoch - 58ms/step
Epoch 12/50
1/1 - 0s - loss: 1612.9438 - val_loss: 1527.6349 

In [16]:
from sklearn.metrics import mean_squared_error

# step 3
# evaluate the model on the test data and compute the mean squared error 
# between the predicted concrete strength and the actual concrete strength
y_pred = model.predict(test_set)

print("Mean squared error: {}".format(mean_squared_error(y_test, y_pred)))

1/1 [==============================] - 0s 178ms/step
Mean squared error: 1504.1938118150783


In [17]:
# step 4
# repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors
mean_squared_errors = []

for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target.to_numpy(), test_size=0.30)
    
    train_set = DatasetGenerator(X_train, y_train, 3507)
    test_set = DatasetGenerator(X_test, y_test, 3507)
    
    model.fit(train_set, validation_data=test_set, epochs=50, verbose=0)
    
    y_pred = model.predict(test_set)
    mean_squared_errors.append(mean_squared_error(y_test, y_pred))

1/1 [==============================] - 0s 37ms/step


In [18]:
# step 5
# report the mean and the standard deviation of the mean squared errors
mse = np.asarray(mean_squared_errors)
mean = mse.mean()
std = mse.std()
print("Mean of the mean squared errors: {}\n".format(mean))
print("Standard deviation of the mean squared errors: {}".format(std))

Mean of the mean squared errors: 488.5810129568092

Standard deviation of the mean squared errors: 456.85069584581635


The mean of the mean squared errors increases compared to that from Step A.